In [1]:
import pandas as pd

In [19]:
df = pd.read_excel("/Users/alicjawojciechowska/Documents/GitHub/ECHA_2025_phase2/PHASE2_inventory_v4032025.xlsx")

In [5]:
df["NAM name"] = df["NAM name"].astype(str).apply(lambda x: x.title())


In [8]:
df["NAM name"] = df["NAM name"].astype(str).apply(lambda x: x.replace('_', ' ').title())

In [12]:
df["Intended purpose "] = df["Intended purpose "].astype(str).apply(lambda x: x.title())

In [14]:
df["Overlap with Phase 1 inventory (yes/no)"] = df["Overlap with Phase 1 inventory (yes/no)"].astype(str).apply(lambda x: x.lower())

In [15]:
df

,NAMs ID,NAM name,Intended purpose,Overlap with Phase 1 inventory (yes/no),Standard Information Requirements (SIRs) / Specific Target Organ Toxicity (STOT),Regulator document reference <lista subsections/Annex z REACH>,Funder,Initiative,Initiative IDs / grant no,Leading institution,Organ/system,"NAMs stage (time period, extpected outcome etc.)","Type (phys-chem characterization, in chemico, in vitro, in silico, ex vivo, in vivo)","Document type (guidance, guideline, ITS, IATA, defined approaches, ISO standards, deliverable, SOP, publication etc.)",Guideline/ strategy,Guideline/ strategy/ source_ref,Nanomaterials used,"test items included conventional chemicals, drugs, vaccines etc.",Unnamed: 18
0,1.0,Particle Size And Size Distribution Of Manufac...,Evaluation Of The Nanopraticle Size And Size D...,yes,Particle size distrubution,REACH: Annex VI subsections 2.3 and 2.4,OECD,OECD Working Party of Manufactured Nanomaterials,-,-,-,validated,physicochemical characterization,test guideline,Test No. 125: Nanomaterial Particle Size and S...,https://www.oecd.org/en/publications/test-no-1...,"[p:68, Table 6] particles: Ag, SiO2 [20nm], Si...",-,NaN
1,2.0,Determination Of Solubility And Dissolution Ra...,Physicochemical Characterization,no,"Water solubility, dissolution rate in relevant...",REACH: Annex VI subsections 2.3 and 2.4,OECD,OECD Working Party of Manufactured Nanomaterials,-,-,-,expected 2025,physicochemical characterization,-,-,-,-,-,NaN
2,3.0,Identification And Quantification Of The Surfa...,Physicochemical Characterization,no,Surface chemistry,REACH: Annex VI subsections 2.3 and 2.4,OECD,OECD Working Party of Manufactured Nanomaterials,-,-,-,expected 2025,physicochemical characterization,-,-,-,-,-,NaN
3,4.0,Determination Of The Dustiness Of Manufactured...,Physicochemical Characterization,no,Dustiness,NaN,OECD,OECD Working Party of Manufactured Nanomaterials,-,-,-,expected 2025,physicochemical characterization,test guideline,Test Guideline on the dustiness determination ...,Draft: https://www.oecd.org/content/dam/oecd/e...,-,-,NaN
4,5.0,Determination Of The (Volume) Specific Surface...,Physicochemical Characterization,no,Surface area,NaN,OECD,NANOMET project,887268,OECD,-,validated,physicochemical characterization,test guideline,Test No. 124: Determination of the Volume Spec...,https://www.oecd.org/en/publications/test-no-1...,"TiO2, SiO2, ZnO",-,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,NaN,Cytotoxicity,The Rtgill-W1 Cell Line Assay Describes A 24-W...,"yes, but not as oecd tg 249 procedure",RTgill-W1 cell,OECD TG 249 / 9.1.3. Short-term toxicity test...,OECD,NaN,NaN,OECD,permanent cell line from rainbow trout (Oncorh...,NaN,in vitro,guidance document,OECD TG 249:\nFish Cell Line Acute Toxicity: T...,https://www.oecd.org/en/publications/test-no-2...,NO,chemicals,NaN
163,NaN,Cytotoxicity Estrogenic Activity,The Easzy Assay Is A Mechanism-Based In Vivo S...,no,estrogen receptors (ERs),OECD TG 250 / 9.1.3. Short-term toxicity test...,OECD,NaN,NaN,OECD,transgenic tg(cyp19a1b:GFP) Zebrafish embryos,NaN,in vivo,guidance document,OECD TG 250: EASZY assay: Detection of Endocri...,https://www.oecd.org/en/publications/test-no-2...,NaN,chemicals,NaN
164,NaN,Cytotoxicity Androgen Disruption Activity,The Rapid Androgen Disruption Activity Reporte...,no,androgen receptore (AR),OECD TG 251 / 9.1.3. Short-term toxicity test...,OECD,NaN,NaN,OECD,transgenic Japanese medaka line harbouring the...,NaN,in vivo,guidance document,OECD TG 251: Rapid Androgen Disruption Activi...,https://www.oecd.org/en/publications/test-no-2...,NaN,chemicals,NaN
165,NaN,Cytotoxicity Estrogen Disruption Activity,This Test Guideline Describes A Rapid Estrogen...,no,estrogen axis,OECD TG 252 / 9.1.3. Short-term toxicity test...,OECD,NaN,NaN,OECD,transgenic Japanese medaka line harbouring the...,NaN,in vivo (embryo),guidance document,OECD TG 252: Rapid Estrogen Activity In Vivo (...,https://www.oecd.org/en/publications/tes

In [18]:
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.worksheet import Worksheet

# Otwieramy plik Excel
wb = load_workbook("/Users/alicjawojciechowska/Documents/GitHub/ECHA_2025_phase2/PHASE2_inventory_v4032025.xlsx")
ws: Worksheet = wb.active

# Znalezienie zakresów do scalania (na podstawie kolumny "Nazwa")
prev_value = None
start_row = 2  # Pomijamy nagłówek

for row in range(2, ws.max_row + 1):
    current_value = ws[f"C{row}"].value  # Kolumna "Nazwa"

    if current_value != prev_value:
        if prev_value is not None:
            ws.merge_cells(f"C{start_row}:C{row-1}")  # Scalanie poprzedniego zakresu
        start_row = row  # Ustawienie nowego punktu startowego

    prev_value = current_value

# Ostatnie scalenie
ws.merge_cells(f"C{start_row}:C{ws.max_row}")

# Zapisujemy plik
wb.save("posortowany_plik_scalony.xlsx")

In [20]:

# Uzupełniamy scalone komórki metodą "forward fill" (przepisywanie wartości w dół)
df.fillna(method='ffill', inplace=True)

# 📌 2. Sortujemy względem kolumny C
df_sorted = df.sort_values(by=df.columns[2], ascending=True)  # Kolumna C (indeks 2)

# 📌 3. Zapisujemy posortowany plik (bez scalania)
temp_file = "posortowany.xlsx"
df_sorted.to_excel(temp_file, index=False, engine='openpyxl')

# 📌 4. Przywracamy scalone komórki
wb = load_workbook(temp_file)
ws = wb.active

def merge_cells(column: str):
    """Scalanie komórek w danej kolumnie, jeśli mają tę samą wartość"""
    prev_value = None
    start_row = 2  # Pomijamy nagłówek

    for row in range(2, ws.max_row + 1):
        current_value = ws[f"{column}{row}"].value

        if current_value != prev_value:
            if prev_value is not None:
                ws.merge_cells(f"{column}{start_row}:{column}{row-1}")  # Scalanie poprzedniego zakresu
            start_row = row

        prev_value = current_value

    ws.merge_cells(f"{column}{start_row}:{column}{ws.max_row}")  # Ostatnie scalenie

# Scalanie kolumn, które były wcześniej scalone
merge_cells("A")  # Jeśli kolumna A była scalona, zmień na właściwą
merge_cells("B")  # Jeśli kolumna B była scalona, zmień na właściwą

# 📌 5. Zapisujemy plik z zachowaniem scalonych komórek
output_file = "posortowany_scalony.xlsx"
wb.save(output_file)

print(f"✅ Plik zapisano jako: {output_file}")

✅ Plik zapisano jako: posortowany_scalony.xlsx


/var/folders/2t/sz6zbhcs41sdvmgklwxf4q0w0000gn/T/ipykernel_11162/732627579.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
